# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-12_14-34-58_18114/logs.
Waiting for redis server at 127.0.0.1:35001 to respond...
Waiting for redis server at 127.0.0.1:17531 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=7fbdae9c001dcdf318cbc08369cccbca47552e3da1baed9c



{'node_ip_address': '172.16.123.117',
 'object_store_addresses': ['/tmp/ray/session_2019-03-12_14-34-58_18114/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-12_14-34-58_18114/sockets/raylet'],
 'redis_address': '172.16.123.117:35001',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=7fbdae9c001dcdf318cbc08369cccbca47552e3da1baed9c'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [64, 32]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy(), 'rl_1': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return agent_id

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })

 Starting SUMO on port 42261


23.196257408802122
3.55673754868485


In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.4/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv-v0_0_2019-03-12_14-35-01ownx4tyw -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_14-36-35
  done: false
  episode_len_mean: 462.0952380952381
  episode_reward_max: 177.99807200266918
  episode_reward_mean: 51.85029512876206
  episode_reward_min: -141.171780641903
  episodes_this_iter: 21
  episodes_total: 21
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 4574.424
    load_time_ms: 159.314
    num_steps_sampled: 10000
    num_ste

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_14-38-15
  done: false
  episode_len_mean: 443.56
  episode_reward_max: 271.2659360999774
  episode_reward_mean: 97.34473175555858
  episode_reward_min: -161.12027829879614
  episodes_this_iter: 23
  episodes_total: 112
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3497.572
    load_time_ms: 33.665
    num_steps_sampled: 50000
    num_steps_trained: 50000
    rl_0:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4250469207763672
      kl: 0.0028582620434463024
      policy_loss: -0.0028415319975465536
      total_loss: 36.61734390258789
      vf_explained_var: 0.40489599108695984
      vf_loss: 36.62015151977539
    rl_1:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.41482675075531
      kl: 0.00896541029214859
      policy_loss: -0.003199500497430563
      t

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 256 s, 9 iter, 90000 ts, 204 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_14-40-27
  done: false
  episode_len_mean: 294.47
  episode_reward_max: 278.89512259050395
  episode_reward_mean: 215.40306944313127
  episode_reward_min: -153.57586899538597
  episodes_this_iter: 36
  episodes_total: 272
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3428.637
    load_time_ms: 18.061
    num_steps_sampled: 100000
    num_steps_trained: 100000
    rl_0:
      cur_kl_coeff: 0.0007812500116415322
      cur_lr: 4.999999873689376e-05
      entropy: 1.417761206626892
      kl: 0.0025372228119522333
      policy_loss: -0.0008997512049973011
      tot

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 379 s, 14 iter, 140000 ts, 197 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_14-42-30
  done: false
  episode_len_mean: 207.46
  episode_reward_max: 275.6967459544674
  episode_reward_mean: 183.66448722667204
  episode_reward_min: -150.69026983776095
  episodes_this_iter: 48
  episodes_total: 498
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3261.901
    load_time_ms: 2.43
    num_steps_sampled: 150000
    num_steps_trained: 150000
    rl_0:
      cur_kl_coeff: 4.882812572759576e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.3759373426437378
      kl: 0.0054731774143874645
      policy_loss: -0.0013969348510727286
      tot

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 506 s, 19 iter, 190000 ts, 235 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_14-44-52
  done: false
  episode_len_mean: 225.37
  episode_reward_max: 276.8713119626218
  episode_reward_mean: 227.00223891505678
  episode_reward_min: -154.72176574546071
  episodes_this_iter: 45
  episodes_total: 715
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3943.722
    load_time_ms: 2.55
    num_steps_sampled: 200000
    num_steps_trained: 200000
    rl_0:
      cur_kl_coeff: 1.5258789289873675e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.395506501197815
      kl: 0.0090343551710248
      policy_loss: -0.003080629510805011
      total_l

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 7426 s, 204 iter, 2040000 ts, 316 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_16-40-10
  done: false
  episode_len_mean: 172.35
  episode_reward_max: 383.1788538710256
  episode_reward_mean: 312.82740402643304
  episode_reward_min: -144.92908031942244
  episodes_this_iter: 58
  episodes_total: 11142
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 4279.38
    load_time_ms: 2.891
    num_steps_sampled: 2050000
    num_steps_trained: 2050000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.15495453774929047
      kl: 0.014566653408110142
      policy_loss: -0.0005287954118102789
      total_loss: 13

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 7615 s, 209 iter, 2090000 ts, 325 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_16-43-25
  done: false
  episode_len_mean: 175.51
  episode_reward_max: 377.53498752304364
  episode_reward_mean: 329.2551001834916
  episode_reward_min: 112.53006708334738
  episodes_this_iter: 57
  episodes_total: 11429
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3944.816
    load_time_ms: 3.116
    num_steps_sampled: 2100000
    num_steps_trained: 2100000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.18886147439479828
      kl: 0.007809900213032961
      policy_loss: -0.00039338826900348067
      total_loss: 2

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 7744 s, 214 iter, 2140000 ts, 313 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_16-45-27
  done: false
  episode_len_mean: 173.66
  episode_reward_max: 388.1348420447826
  episode_reward_mean: 314.8393066905221
  episode_reward_min: -150.01076892416776
  episodes_this_iter: 58
  episodes_total: 11718
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3922.045
    load_time_ms: 2.993
    num_steps_sampled: 2150000
    num_steps_trained: 2150000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.18521782755851746
      kl: 0.011339694261550903
      policy_loss: -3.682974784169346e-05
      total_loss: 82

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 7891 s, 219 iter, 2190000 ts, 332 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_16-47-59
  done: false
  episode_len_mean: 176.24
  episode_reward_max: 375.9326115120743
  episode_reward_mean: 328.6385660795962
  episode_reward_min: -150.42253352297146
  episodes_this_iter: 56
  episodes_total: 12002
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3634.437
    load_time_ms: 2.428
    num_steps_sampled: 2200000
    num_steps_trained: 2200000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.1834312379360199
      kl: 0.030773339793086052
      policy_loss: -0.0023920638486742973
      total_loss: 67.

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 8.0/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=18174], 8020 s, 224 iter, 2240000 ts, 315 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-12_16-50-02
  done: false
  episode_len_mean: 173.79
  episode_reward_max: 381.5565466393143
  episode_reward_mean: 310.1378297934546
  episode_reward_min: -143.49300245252738
  episodes_this_iter: 58
  episodes_total: 12287
  experiment_id: abc197f3c9fd4810a90b9d1fdf241416
  hostname: Gandalf
  info:
    grad_time_ms: 3667.022
    load_time_ms: 2.527
    num_steps_sampled: 2250000
    num_steps_trained: 2250000
    rl_0:
      cur_kl_coeff: 0.0
      cur_lr: 4.999999873689376e-05
      entropy: 0.12315817177295685
      kl: 0.020366599783301353
      policy_loss: -0.0004213926149532199
      total_loss: 18